# **1. Extract information about Justin Trudeau for sysdate-1**

In [39]:
import pandas as pd
import requests
import datetime
import urllib.parse
from datetime import datetime
from datetime import timedelta

In [42]:
parameters={
    'from-date' : '2016-01-01',
    'to-date' : str(datetime.today())[:10],
    'q' : 'Justin%20AND%20Trudeau%20OR%20JustinTrudeau',
    'format' : 'json',
    'page' : 1,
    'page-size' : 10,
    'api-key' : "3a6cb1a7-fb92-4191-b819-fd230c164b43"
}

main_url='https://content.guardianapis.com/search?'



def extractData(params, main_urladdress):
    url=main_urladdress+ urllib.parse.urlencode(params).replace('%2520', '%20')
    data= requests.get(url).json()
    if data['response']['status']=='ok':
        return data

In [43]:
fullData = extractData(parameters,main_url)

numberofResults= fullData['response']['total']
numberofPages= fullData['response']['pages']
print('Number of results: {}'.format(numberofResults))
print('Number of pages: {}'.format(numberofPages))

Number of results: 1899
Number of pages: 190


In [44]:
def appendfullData(params, numPages, fullData):
    for currentpage in range(2, numPages+1):
        parameters['page']= currentpage
        currentData = extractData(parameters,main_url)
        if currentData['response']['results'] is not None:
            fullData['response']['results'].extend(currentData['response']['results'])
        else:
            continue

In [45]:
appendfullData(parameters, numberofPages, fullData)

In [89]:
def getTableofData(data):
    i=1
    outputDict= {}
    for results in data['response']['results']:
        if results['webPublicationDate'] is not None and results['type'] is not None and results['sectionName'] is not None:
            temp1=str(results['webPublicationDate'])
            temp2=str(results['type'])
            temp3=str(results['sectionName'])
            temp4=str(results["webUrl"])
            temp5=str(results["sectionId"])
            temp6=str(results["id"])
            if i==1:
                outputDict= {'dateandType': [[temp1,temp2,temp3,temp4,temp5,temp6]]}
            else:
                outputDict['dateandType'].append([temp1,temp2,temp3,temp4,temp5,temp6])
            i=i+1
    
    labels = ['Date', 'Type', 'section','webUrl','sectionId','id']
    dataSubset=pd.DataFrame.from_records(outputDict['dateandType'], columns=labels)
    
    return dataSubset

In [90]:
filteredData = getTableofData(fullData)

In [93]:
def convertDatefield(data):
    data = data.apply(lambda x: x[0:10])
    data = pd.to_datetime(data)
    return data

In [94]:
filteredData['Date'] = convertDatefield(filteredData['Date'])

In [95]:
filteredData

,Date,Type,section,webUrl,sectionId,id
0,2023-02-08,article,World news,https://www.theguardian.com/world/2023/feb/08/...,world,world/2023/feb/08/canadas-justin-trudeau-greet...
1,2023-03-07,article,World news,https://www.theguardian.com/world/2023/mar/07/...,world,world/2023/mar/07/canada-pm-justin-trudeau-spe...
2,2022-12-08,article,World news,https://www.theguardian.com/world/2022/dec/08/...,world,world/2022/dec/08/sovereignty-act-passed-alber...
3,2023-03-24,article,World news,https://www.theguardian.com/world/2023/mar/24/...,world,world/2023/mar/24/trudeau-biden-asylum-border-...
4,2022-11-02,article,Television & radio,https://www.theguardian.com/tv-and-radio/2022/...,tv-and-radio,tv-and-radio/2022/nov/02/drugs-impotence-and-p...
...,...,...,...,...,...,...
1894,2016-11-10,liveblog,US news,https://www.theguardian.com/us-news/live/2016/...,us-news,us-news/live/2016/nov/09/donald-trump-us-elect...
1895,2016-05-12,liveblog,Australia news,https://www.theguardian.com/australia-news/liv...,australia-news,australia-news/live/2016/may/12/australian-ele...
1896,2016-07-13,liveblog,Politics,https://www.theguardian.com/politics/blog/live...,politics,politics/blog/live/2016/jul/13/pmqs-cameron-ma...
1897,2016-06-17,liveblog,UK news,https://www.theguardian.com/politics/live/2016...,uk-news,politics/live/2016/jun/16/eu-referendum-live-o...


# **2. Save the result as a file in the local folder**

In [92]:
filteredData.to_csv(r"C:\Users\Lion\Desktop\Test.csv",sep=",")

# **3.Read the file and import it into the database using “Bulk Insert”**